In [1]:
import warnings
from collections import Counter
import numpy as np
import pandas as pd

### Subreddit Data
The df dataframe was built specifically for Direct graph

In [2]:
df = pd.read_csv("scraping data/data/data_subreddit_direct.csv",index_col=0)
df.head()

,parent,to,interactions,crossposts,date,weights
0,nature,environment,1746.0,1,1.619228e+09,7.465655
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,5.043425
2,nature,PublicLands,3.0,1,1.619177e+09,1.386294
3,nature,EndangeredSpecies,28.0,2,1.618449e+09,3.367296
4,nature,animalimpact,5.0,3,1.615263e+09,1.791759


# Build dataframe "df1" for Unditected graph

In [3]:
df1=df.copy()
#delete the columns 
columns2remove = ['weights']
df1.drop(columns2remove, inplace=True, axis=1)
df1.head()

,parent,to,interactions,crossposts,date
0,nature,environment,1746.0,1,1.619228e+09
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09
2,nature,PublicLands,3.0,1,1.619177e+09
3,nature,EndangeredSpecies,28.0,2,1.618449e+09
4,nature,animalimpact,5.0,3,1.615263e+09


Made df1  that is the copy of df with the addition of the "index1" column.<br>
Made made df2 which is the copy of df1 where the column name "parent" has been changed to "to" and the column name "to" to "parent" and "index1" to "index2".

In [4]:
id=range(len(df1))
df1["index1"]=id
df1.head()

,parent,to,interactions,crossposts,date,index1
0,nature,environment,1746.0,1,1.619228e+09,0
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1
2,nature,PublicLands,3.0,1,1.619177e+09,2
3,nature,EndangeredSpecies,28.0,2,1.618449e+09,3
4,nature,animalimpact,5.0,3,1.615263e+09,4


In [5]:
df.loc[df1['parent']== df1['to']]


,parent,to,interactions,crossposts,date,weights


In [6]:
df2=df1.copy()
df2.rename(columns = {'to':'to2', 'parent':'to'}, inplace = True)
df2.rename(columns = {'to2':'parent','index1':'index2'}, inplace = True)

df2.head()

,to,parent,interactions,crossposts,date,index2
0,nature,environment,1746.0,1,1.619228e+09,0
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1
2,nature,PublicLands,3.0,1,1.619177e+09,2
3,nature,EndangeredSpecies,28.0,2,1.618449e+09,3
4,nature,animalimpact,5.0,3,1.615263e+09,4


Now let's look for the rows in df1 and df2 that have the "parent" and "to" subredits equal at the same time.

In [7]:
result=[]
result=pd.merge(df1,df2,on=["parent","to"], how="inner")
result.head()

,parent,to,interactions_x,crossposts_x,date_x,index1,interactions_y,crossposts_y,date_y,index2
0,nature,environment,1746.0,1,1.619228e+09,0,219.0,2,1.609549e+09,101
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1,138.0,5,1.618086e+09,26779
2,nature,UpliftingNews,51.0,1,1.617663e+09,10,31.0,2,1.600145e+09,3973
3,WomenInNews,WomensRightsNews,64.0,4,1.614272e+09,19,8.0,1,1.614636e+09,54801
4,environment,EcoNewsNetwork,412.0,43,1.556766e+09,31,2582.0,24,1.603487e+09,26775


In [8]:
result1=result.copy()
result1.head()

,parent,to,interactions_x,crossposts_x,date_x,index1,interactions_y,crossposts_y,date_y,index2
0,nature,environment,1746.0,1,1.619228e+09,0,219.0,2,1.609549e+09,101
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1,138.0,5,1.618086e+09,26779
2,nature,UpliftingNews,51.0,1,1.617663e+09,10,31.0,2,1.600145e+09,3973
3,WomenInNews,WomensRightsNews,64.0,4,1.614272e+09,19,8.0,1,1.614636e+09,54801
4,environment,EcoNewsNetwork,412.0,43,1.556766e+09,31,2582.0,24,1.603487e+09,26775


I create the "interactions" and "crossposts" columns, adding inteactions_x + interaction_y and crosspost_x + crosspost_y

In [9]:
result["interactions"]=result["interactions_x"]+result["interactions_y"]
result["crossposts"]=result["crossposts_x"]+result["crossposts_y"]
result.head()    


,parent,to,interactions_x,crossposts_x,date_x,index1,interactions_y,crossposts_y,date_y,index2,interactions,crossposts
0,nature,environment,1746.0,1,1.619228e+09,0,219.0,2,1.609549e+09,101,1965.0,3
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1,138.0,5,1.618086e+09,26779,292.0,18
2,nature,UpliftingNews,51.0,1,1.617663e+09,10,31.0,2,1.600145e+09,3973,82.0,3
3,WomenInNews,WomensRightsNews,64.0,4,1.614272e+09,19,8.0,1,1.614636e+09,54801,72.0,5
4,environment,EcoNewsNetwork,412.0,43,1.556766e+09,31,2582.0,24,1.603487e+09,26775,2994.0,67


I create a new "date" attribute: between date_x and date_y, "date" is the smallest value and represents the oldest crosspost date

In [10]:
date=[]
for i in range(len(result)):
    x=[]
    x=min([result['date_x'][i],result['date_y'][i]])
    date.append(x)
    
result['date']=date
result.head()

,parent,to,interactions_x,crossposts_x,date_x,index1,interactions_y,crossposts_y,date_y,index2,interactions,crossposts,date
0,nature,environment,1746.0,1,1.619228e+09,0,219.0,2,1.609549e+09,101,1965.0,3,1.609549e+09
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1,138.0,5,1.618086e+09,26779,292.0,18,1.615282e+09
2,nature,UpliftingNews,51.0,1,1.617663e+09,10,31.0,2,1.600145e+09,3973,82.0,3,1.600145e+09
3,WomenInNews,WomensRightsNews,64.0,4,1.614272e+09,19,8.0,1,1.614636e+09,54801,72.0,5,1.614272e+09
4,environment,EcoNewsNetwork,412.0,43,1.556766e+09,31,2582.0,24,1.603487e+09,26775,2994.0,67,1.556766e+09


Now we are going to replace the values ​​of "interactions", "crossposts" and "date", in the rows with "index1" equal to indexes of the dataframe df1.<br>
First we sort the dataframe "result", by "index1"

In [11]:
result = result.sort_values(by = 'index1', ascending = True)
result.head()

,parent,to,interactions_x,crossposts_x,date_x,index1,interactions_y,crossposts_y,date_y,index2,interactions,crossposts,date
0,nature,environment,1746.0,1,1.619228e+09,0,219.0,2,1.609549e+09,101,1965.0,3,1.609549e+09
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1,138.0,5,1.618086e+09,26779,292.0,18,1.615282e+09
2,nature,UpliftingNews,51.0,1,1.617663e+09,10,31.0,2,1.600145e+09,3973,82.0,3,1.600145e+09
3,WomenInNews,WomensRightsNews,64.0,4,1.614272e+09,19,8.0,1,1.614636e+09,54801,72.0,5,1.614272e+09
4,environment,EcoNewsNetwork,412.0,43,1.556766e+09,31,2582.0,24,1.603487e+09,26775,2994.0,67,1.556766e+09


In [12]:
df1.head()

,parent,to,interactions,crossposts,date,index1
0,nature,environment,1746.0,1,1.619228e+09,0
1,nature,EcoNewsNetwork,154.0,13,1.615282e+09,1
2,nature,PublicLands,3.0,1,1.619177e+09,2
3,nature,EndangeredSpecies,28.0,2,1.618449e+09,3
4,nature,animalimpact,5.0,3,1.615263e+09,4


I'm going to insert the new values ​​of the attributes "Interactions", "crosspost" and "date" in the corresponding rows of the dataframe df1, that is, in the respective pairs of subreddit "from" and "to"

In [13]:
for i in range(len(result)):
    x=[]
    x=result['index1'].iloc[i]
    df1['interactions'].iloc[x]=result['interactions'].iloc[i]
    df1['crossposts'].iloc[x]=result['crossposts'].iloc[i]
    df1['date'].iloc[x]=result['date'].iloc[i]
    

C:\Users\cinzia\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
df1.head()

,parent,to,interactions,crossposts,date,index1
0,nature,environment,1965.0,3,1.609549e+09,0
1,nature,EcoNewsNetwork,292.0,18,1.615282e+09,1
2,nature,PublicLands,3.0,1,1.619177e+09,2
3,nature,EndangeredSpecies,28.0,2,1.618449e+09,3
4,nature,animalimpact,5.0,3,1.615263e+09,4


Second step we delete the rows of df1 ,with index "index2" of dataframe "result"<br>
We delete the rows from the dataframe df1 starting from the major index to the minor index, starting from the end of df1.<br>
In this way the index of the rows not yet deleted will not be changed.

In [15]:
result = result.sort_values(by = 'index2', ascending = False)
result.head()


,parent,to,interactions_x,crossposts_x,date_x,index1,interactions_y,crossposts_y,date_y,index2,interactions,crossposts,date
4998,Chodi,IndiansRead,12.0,1,1.619295e+09,99518,25.0,2,1.618342e+09,107073,37.0,3,1.618342e+09
3479,opensourcehardware,RISCV,26.0,1,1.576024e+09,76930,13.0,1,1.595022e+09,107037,39.0,2,1.576024e+09
4401,opensource,openhardware,14.0,1,1.581452e+09,90098,3.0,1,1.615095e+09,107027,17.0,2,1.581452e+09
1634,aww,goats,513.0,5,1.594784e+09,45543,13.0,1,1.619220e+09,107004,526.0,6,1.594784e+09
1967,PeopleFuckingDying,ADailyDoseOfInternet,60.0,1,1.599193e+09,53359,16.0,1,1.588820e+09,106998,76.0,2,1.588820e+09


In [16]:
#delete the columns 
columns2remove = ['interactions_x','interactions_y','crossposts_x','crossposts_y','date_x','date_y']
result.drop(columns2remove, inplace=True, axis=1)
result.head()


,parent,to,index1,index2,interactions,crossposts,date
4998,Chodi,IndiansRead,99518,107073,37.0,3,1.618342e+09
3479,opensourcehardware,RISCV,76930,107037,39.0,2,1.576024e+09
4401,opensource,openhardware,90098,107027,17.0,2,1.581452e+09
1634,aww,goats,45543,107004,526.0,6,1.594784e+09
1967,PeopleFuckingDying,ADailyDoseOfInternet,53359,106998,76.0,2,1.588820e+09


In [17]:
result2=result.copy()
result2.head()

,parent,to,index1,index2,interactions,crossposts,date
4998,Chodi,IndiansRead,99518,107073,37.0,3,1.618342e+09
3479,opensourcehardware,RISCV,76930,107037,39.0,2,1.576024e+09
4401,opensource,openhardware,90098,107027,17.0,2,1.581452e+09
1634,aww,goats,45543,107004,526.0,6,1.594784e+09
1967,PeopleFuckingDying,ADailyDoseOfInternet,53359,106998,76.0,2,1.588820e+09


In [18]:
for i in range(len(result)): 
    
    for j in range(len(result)):
        if (result['parent'][i]==result['to'][j]) & (result['to'][i]==result['parent'][j]):
            print('row=',i,'row change =',j)
            result.loc[j]=result.loc[i]
            break
            
       
              
            
    

row= 0 row change = 19
row= 1 row change = 956
row= 2 row change = 117
row= 3 row change = 2039
row= 4 row change = 954
row= 5 row change = 762
row= 6 row change = 1553
row= 7 row change = 620
row= 8 row change = 2574
row= 9 row change = 853
row= 10 row change = 2629
row= 11 row change = 2636
row= 12 row change = 2674
row= 13 row change = 3258
row= 14 row change = 3264
row= 15 row change = 2671
row= 16 row change = 3134
row= 17 row change = 2884
row= 18 row change = 468
row= 20 row change = 5120
row= 21 row change = 3288
row= 22 row change = 780
row= 23 row change = 3110
row= 24 row change = 477
row= 25 row change = 5469
row= 26 row change = 5077
row= 27 row change = 3296
row= 28 row change = 638
row= 29 row change = 4015
row= 30 row change = 3226
row= 31 row change = 3221
row= 32 row change = 1152
row= 33 row change = 4224
row= 34 row change = 4470
row= 35 row change = 43
row= 36 row change = 5389
row= 37 row change = 3261
row= 38 row change = 1371
row= 39 row change = 1117
row= 40 ro

row= 325 row change = 1339
row= 326 row change = 4409
row= 327 row change = 2872
row= 328 row change = 790
row= 329 row change = 4225
row= 330 row change = 1130
row= 331 row change = 2042
row= 332 row change = 922
row= 333 row change = 3827
row= 334 row change = 505
row= 335 row change = 2811
row= 336 row change = 3001
row= 337 row change = 2971
row= 339 row change = 3957
row= 340 row change = 3824
row= 341 row change = 1203
row= 342 row change = 2818
row= 343 row change = 3009
row= 344 row change = 4243
row= 345 row change = 2711
row= 346 row change = 4324
row= 347 row change = 1443
row= 348 row change = 2916
row= 349 row change = 562
row= 350 row change = 1551
row= 351 row change = 1487
row= 352 row change = 5042
row= 353 row change = 2983
row= 355 row change = 1334
row= 356 row change = 4130
row= 358 row change = 788
row= 359 row change = 1156
row= 360 row change = 1493
row= 361 row change = 1546
row= 362 row change = 2837
row= 363 row change = 3008
row= 364 row change = 1354
row= 3

row= 689 row change = 1478
row= 691 row change = 1483
row= 692 row change = 1400
row= 693 row change = 2044
row= 694 row change = 1124
row= 695 row change = 2556
row= 697 row change = 2904
row= 699 row change = 1396
row= 702 row change = 2895
row= 703 row change = 5195
row= 704 row change = 5011
row= 705 row change = 1378
row= 706 row change = 984
row= 707 row change = 995
row= 708 row change = 3386
row= 709 row change = 3392
row= 710 row change = 749
row= 711 row change = 3415
row= 712 row change = 2660
row= 713 row change = 2633
row= 714 row change = 3436
row= 715 row change = 2626
row= 716 row change = 3661
row= 717 row change = 5511
row= 718 row change = 2667
row= 719 row change = 5513
row= 720 row change = 3413
row= 721 row change = 5100
row= 722 row change = 2625
row= 723 row change = 3660
row= 724 row change = 3446
row= 726 row change = 3406
row= 727 row change = 5108
row= 730 row change = 2619
row= 731 row change = 1173
row= 732 row change = 5097
row= 733 row change = 944
row= 

row= 1080 row change = 2480
row= 1081 row change = 4175
row= 1082 row change = 4131
row= 1083 row change = 1479
row= 1085 row change = 4240
row= 1086 row change = 5161
row= 1087 row change = 2300
row= 1088 row change = 3014
row= 1089 row change = 3897
row= 1090 row change = 4407
row= 1091 row change = 4166
row= 1093 row change = 3395
row= 1094 row change = 5231
row= 1097 row change = 1222
row= 1099 row change = 4292
row= 1100 row change = 2736
row= 1102 row change = 1717
row= 1103 row change = 1224
row= 1105 row change = 1415
row= 1106 row change = 2929
row= 1107 row change = 2603
row= 1108 row change = 1289
row= 1110 row change = 1410
row= 1111 row change = 3777
row= 1112 row change = 1380
row= 1115 row change = 3986
row= 1119 row change = 4145
row= 1120 row change = 4333
row= 1121 row change = 3908
row= 1123 row change = 5604
row= 1125 row change = 1481
row= 1127 row change = 2920
row= 1128 row change = 3979
row= 1129 row change = 2566
row= 1131 row change = 4012
row= 1132 row change

row= 1527 row change = 3495
row= 1528 row change = 4406
row= 1530 row change = 4325
row= 1537 row change = 3825
row= 1541 row change = 5146
row= 1543 row change = 5212
row= 1549 row change = 2718
row= 1554 row change = 4425
row= 1555 row change = 3086
row= 1556 row change = 3339
row= 1557 row change = 5305
row= 1560 row change = 4269
row= 1561 row change = 2015
row= 1562 row change = 2329
row= 1563 row change = 3812
row= 1564 row change = 4018
row= 1566 row change = 4968
row= 1567 row change = 2381
row= 1568 row change = 2778
row= 1569 row change = 2783
row= 1570 row change = 2389
row= 1571 row change = 3725
row= 1572 row change = 4351
row= 1573 row change = 4179
row= 1574 row change = 2790
row= 1576 row change = 3844
row= 1577 row change = 2349
row= 1578 row change = 2351
row= 1579 row change = 4957
row= 1580 row change = 4960
row= 1581 row change = 2399
row= 1583 row change = 5152
row= 1585 row change = 2883
row= 1586 row change = 2914
row= 1588 row change = 2758
row= 1589 row change

row= 1912 row change = 4080
row= 1913 row change = 1977
row= 1914 row change = 3631
row= 1915 row change = 4289
row= 1916 row change = 2199
row= 1919 row change = 2115
row= 1920 row change = 3840
row= 1921 row change = 4381
row= 1922 row change = 3365
row= 1923 row change = 4556
row= 1924 row change = 2218
row= 1925 row change = 3805
row= 1926 row change = 1965
row= 1928 row change = 2443
row= 1929 row change = 2096
row= 1930 row change = 4880
row= 1931 row change = 4570
row= 1932 row change = 3311
row= 1934 row change = 4636
row= 1935 row change = 2018
row= 1936 row change = 4086
row= 1937 row change = 2024
row= 1938 row change = 3350
row= 1939 row change = 3855
row= 1940 row change = 4868
row= 1941 row change = 4780
row= 1943 row change = 2645
row= 1945 row change = 2561
row= 1948 row change = 3671
row= 1950 row change = 2431
row= 1951 row change = 3589
row= 1952 row change = 3662
row= 1953 row change = 4886
row= 1955 row change = 3352
row= 1956 row change = 3701
row= 1957 row change

row= 2387 row change = 4152
row= 2388 row change = 2723
row= 2390 row change = 3998
row= 2394 row change = 3018
row= 2397 row change = 4247
row= 2402 row change = 4969
row= 2403 row change = 4970
row= 2405 row change = 4501
row= 2407 row change = 3581
row= 2408 row change = 3585
row= 2409 row change = 4440
row= 2410 row change = 3095
row= 2411 row change = 4533
row= 2412 row change = 4554
row= 2414 row change = 5537
row= 2415 row change = 4975
row= 2416 row change = 4696
row= 2417 row change = 3648
row= 2418 row change = 4633
row= 2419 row change = 4827
row= 2420 row change = 4890
row= 2422 row change = 5170
row= 2423 row change = 3333
row= 2424 row change = 4525
row= 2425 row change = 4918
row= 2429 row change = 4893
row= 2432 row change = 3318
row= 2437 row change = 3315
row= 2441 row change = 2510
row= 2444 row change = 4783
row= 2446 row change = 3695
row= 2447 row change = 4120
row= 2452 row change = 5219
row= 2453 row change = 2845
row= 2454 row change = 4167
row= 2455 row change

row= 2998 row change = 3006
row= 3003 row change = 4477
row= 3015 row change = 4436
row= 3016 row change = 5209
row= 3020 row change = 4370
row= 3021 row change = 4481
row= 3022 row change = 3710
row= 3023 row change = 3707
row= 3024 row change = 4368
row= 3026 row change = 4483
row= 3028 row change = 5474
row= 3030 row change = 5000
row= 3031 row change = 5287
row= 3032 row change = 5072
row= 3033 row change = 4928
row= 3034 row change = 4934
row= 3035 row change = 5256
row= 3037 row change = 5259
row= 3038 row change = 4933
row= 3039 row change = 3040
row= 3042 row change = 5241
row= 3043 row change = 5265
row= 3044 row change = 4955
row= 3045 row change = 5071
row= 3046 row change = 4926
row= 3047 row change = 3877
row= 3049 row change = 4740
row= 3050 row change = 3879
row= 3053 row change = 4925
row= 3055 row change = 5264
row= 3057 row change = 5282
row= 3058 row change = 5262
row= 3059 row change = 5276
row= 3061 row change = 3063
row= 3062 row change = 3064
row= 3068 row change

row= 3627 row change = 4161
row= 3628 row change = 3854
row= 3634 row change = 5037
row= 3635 row change = 4393
row= 3636 row change = 5295
row= 3637 row change = 4885
row= 3639 row change = 4541
row= 3640 row change = 4860
row= 3641 row change = 4546
row= 3643 row change = 4974
row= 3645 row change = 4555
row= 3646 row change = 4526
row= 3647 row change = 5540
row= 3650 row change = 4692
row= 3651 row change = 3652
row= 3653 row change = 4820
row= 3655 row change = 4457
row= 3656 row change = 5491
row= 3657 row change = 5234
row= 3665 row change = 5518
row= 3666 row change = 4985
row= 3668 row change = 4615
row= 3672 row change = 4377
row= 3675 row change = 4104
row= 3676 row change = 3679
row= 3680 row change = 5311
row= 3683 row change = 3704
row= 3685 row change = 4098
row= 3690 row change = 3794
row= 3698 row change = 4667
row= 3717 row change = 4578
row= 3720 row change = 4375
row= 3723 row change = 3884
row= 3726 row change = 4752
row= 3727 row change = 5211
row= 3732 row change

row= 4770 row change = 4774
row= 4772 row change = 5045
row= 4773 row change = 5222
row= 4777 row change = 5570
row= 4779 row change = 4873
row= 4788 row change = 5614
row= 4793 row change = 5248
row= 4794 row change = 5254
row= 4795 row change = 4796
row= 4807 row change = 5545
row= 4812 row change = 5065
row= 4813 row change = 5266
row= 4814 row change = 4835
row= 4821 row change = 4825
row= 4828 row change = 5501
row= 4839 row change = 5488
row= 4844 row change = 4845
row= 4848 row change = 4849
row= 4855 row change = 4856
row= 4859 row change = 5496
row= 4877 row change = 5356
row= 4878 row change = 4879
row= 4882 row change = 5572
row= 4897 row change = 4898
row= 4899 row change = 4902
row= 4901 row change = 5296
row= 4903 row change = 4904
row= 4916 row change = 5260
row= 4920 row change = 5068
row= 4923 row change = 5258
row= 4929 row change = 5073
row= 4931 row change = 5067
row= 4935 row change = 5257
row= 4940 row change = 5414
row= 4941 row change = 4942
row= 4944 row change

In [19]:
result2=result.copy()
result2.head()

,parent,to,index1,index2,interactions,crossposts,date
4998,Chodi,IndiansRead,99518,107073,37.0,3,1.618342e+09
3479,opensourcehardware,RISCV,76930,107037,39.0,2,1.576024e+09
4401,opensource,openhardware,90098,107027,17.0,2,1.581452e+09
1634,aww,goats,45543,107004,526.0,6,1.594784e+09
1967,PeopleFuckingDying,ADailyDoseOfInternet,53359,106998,76.0,2,1.588820e+09


In [20]:
result = result.sort_values(by = 'parent', ascending = True)
result.head()
size_before=len(result)
print('Size "result" before drop duplicate rows is =',size_before)
result.drop_duplicates(keep="first",inplace=True)
size_after=len(result)    

print('Size result After drop rows is =',size_after) 
print('we have deletes "', size_before-size_after,'" duplicate rows')



Size "result" before drop duplicate rows is = 5620
Size result After drop rows is = 2810
we have deletes " 2810 " duplicate rows


In [21]:

size_before=len(df1)
print('Size df1 before drop rows is =',size_before)
df1=df1.drop(result['index2'])
size_after=len(df1)    

print('Size df1 After drop rows is =',size_after) 
print('we have deletes "', size_before-size_after,'" rows')


Size df1 before drop rows is = 107094
Size df1 After drop rows is = 104284
we have deletes " 2810 " rows


In [22]:
#reset the indexes in the new DataFrame
df1.reset_index(inplace=True,drop=True)
df1.head()

,parent,to,interactions,crossposts,date,index1
0,nature,environment,1965.0,3,1.609549e+09,0
1,nature,EcoNewsNetwork,292.0,18,1.615282e+09,1
2,nature,PublicLands,3.0,1,1.619177e+09,2
3,nature,EndangeredSpecies,28.0,2,1.618449e+09,3
4,nature,animalimpact,5.0,3,1.615263e+09,4


In [23]:

i=0
index=[]
for i in range(len(df1)-1):
    if df1['parent'].iloc[i]==df1['parent'].iloc[i+1] and df1['to'].iloc[i]==df1['to'].iloc[i+1]:
        index.append([i,i+1])
    
if len(index)==0:
    print('There are no duplicates of rows with parent and to equal to other rows')
else:
    print('There rows with duplicates of  "parent" and "to" ')
  

There are no duplicates of rows with parent and to equal to other rows


I delete the "index1" column from df1 and add the new "weights" attribute

In [24]:
df1["weights"] = np.log(df1["interactions"]+1)
#delete the columns 
columns2remove = ['index1']
df1.drop(columns2remove, inplace=True, axis=1)
df1.head()


,parent,to,interactions,crossposts,date,weights
0,nature,environment,1965.0,3,1.609549e+09,7.583756
1,nature,EcoNewsNetwork,292.0,18,1.615282e+09,5.680173
2,nature,PublicLands,3.0,1,1.619177e+09,1.386294
3,nature,EndangeredSpecies,28.0,2,1.618449e+09,3.367296
4,nature,animalimpact,5.0,3,1.615263e+09,1.791759


In [26]:
df1.to_csv("scraping data/data/data_subreddit_Undirect.csv")